In [2]:
# | default_exp routes.application

In [3]:
# | exporti
from typing import Union
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

from pprint import pprint

In [4]:
# | hide
import os
import pandas as pd

In [5]:
# | export


class ApplicationError_NoneRetrieved(de.DomoError):
    def __init__(
        self,
        domo_instance,
        application_id=None,
        application_name=None,
        parent_class=None,
        function_name=None,
        job_id=None,
    ):
        message = "No applications retrieve"

        if application_id:
            message = f"unable to retrieve application - {application_id}"

        if application_name:
            message = f"unable to retrieve application - {application_name}"

        if application_id and job_id:
            message = (
                f"unable to retrieve {job_id} job from application - {application_name}"
            )

        super().__init__(
            message=message,
            parent_class=parent_class,
            function_name=function_name,
            domo_instance=domo_instance,
        )


@gd.route_function
async def get_applications(
    auth: dmda.DomoAuth,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/executor/v1/applications/"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success:
        raise ApplicationError_NoneRetrieved(
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )
    return res

#### sample implementation of get_applications


In [6]:
#| eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_applications(auth=token_auth)
pd.DataFrame(res.response)[0:5]

warning this token has not been validated by who_am_i, run get_auth_token first


,applicationId,customerId,name,description,version,executionClass,verifyOwnership,strictValidation,imageLocation,authorities,internalAuthorities,created,updated,resources,k8s
0,25a97e0c-df6b-11eb-ba80-0242ac130004,mmmm-0012-0200,Toolkit: PDP Automation,PDP Automation,1.0.102_1,com.domo.executor.pdp.PDPAutomationApplication,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,"[pipeline.executor.job.create, dataset.manage]",[],1665079183000,1705595224000,"{'requests': {'memory': '3G', 'missingMemoryVa...",True
1,4ddbf5d7-6441-4eb3-b5aa-97707cae2d2b,mmmm-0012-0200,Toolkit: Schema Management,Schema Management,1.0.25_3,com.domo.executor.schema.SchemaManagement,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,"[pipeline.executor.job.create, dataset.admin]",[],1665079188000,1701972110000,"{'requests': {'memory': '2G', 'missingMemoryVa...",True
2,e37a6942-9c0f-485a-8288-4fe95e10b23d,mmmm-0012-0200,Toolkit: DataSet S3 Backup,DataSet S3 Backup,1.0.7_2,com.domo.executor.backup.DataSetBackupExecutor,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,"[dataset.admin, search.allentities.query]",[service.internal],1665079176000,1701972066000,"{'requests': {'memory': '3G', 'missingMemoryVa...",True
3,b52f3c80-2642-4dcb-b874-b327326021b0,mmmm-0012-0200,Toolkit: User Automation,Toolkit: User Automation,1.80_1,com.domo.executor.userautomation.UserAutomatio...,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,"[pipeline.executor.job.create, dataset.manage,...",[],1665079192000,1707433871000,"{'requests': {'memory': '3G', 'missingMemoryVa...",True
4,2f6573a5-97d8-4e27-b0fd-3c0f2313a3c8,mmmm-0012-0200,Toolkit: Group Management,Group Management,1.0.43_2,com.domo.executor.automation.GroupAutomationEx...,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,"[pipeline.executor.job.create, dataset.admin, ...",[],1665079181000,1701972084000,"{'requests': {'memory': '1G', 'missingMemoryVa...",True


In [7]:
# | export


@gd.route_function
async def get_application_by_id(
    auth: dmda.DomoAuth,
    application_id: str,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success:
        raise ApplicationError_NoneRetrieved(
            domo_instance=auth.domo_instance,
            application_id=application_id,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [8]:
#| eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await get_applications(auth=token_auth)
application = next(
    (app for app in res.response if app["name"] == "Toolkit: DataSet Tag Automation")
)

res = await get_application_by_id(
    application_id=application["applicationId"], auth=token_auth
)
res.response

warning this token has not been validated by who_am_i, run get_auth_token first


{'applicationId': 'a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4',
 'customerId': 'mmmm-0012-0200',
 'name': 'Toolkit: DataSet Tag Automation',
 'description': 'DataSet Tag Automation',
 'version': '1.0.35_2',
 'executionClass': 'com.domo.executor.tags.ResourceTagApplication',
 'verifyOwnership': False,
 'strictValidation': True,
 'imageLocation': '972900829293.dkr.ecr.us-east-1.amazonaws.com/executor/resourcetag-automation:1.0.35_master',
 'authorities': ['dataflow.admin',
  'dataset.admin',
  'search.allentities.query'],
 'internalAuthorities': [],
 'created': 1665079179000,
 'updated': 1701972075000,
 'resources': {'requests': {'memory': '3G', 'missingMemoryValue': False},
  'limits': {'memory': '3G', 'missingMemoryValue': False},
  'missingMemoryRequestValue': False,
  'missingMemoryLimitValue': False},
 'k8s': True}

# Jobs


In [9]:
# | export


class ApplicationError_NoJobRetrieved(de.DomoError):
    def __init__(
        self,
        domo_instance,
        application_id=None,
        parent_class=None,
        function_name=None,
    ):
        message = f"no jobs retrieved from application - {application_id}"

        super().__init__(
            message=message,
            parent_class=parent_class,
            function_name=function_name,
            domo_instance=domo_instance,
        )


gd.route_function


async def get_application_jobs(
    auth: dmda.DomoFullAuth,
    application_id: str,
    parent_class: str = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
) -> rgd.ResponseGetData:

    offset_params = {"offset": "offset", "limit": "limit"}

    url = f"https://{auth.domo_instance}.domo.com/api/executor/v2/applications/{application_id}/jobs"

    def arr_fn(res) -> list[dict]:
        return res.response.get("jobs")

    res = await gd.looper(
        auth=auth,
        method="GET",
        url=url,
        arr_fn=arr_fn,
        loop_until_end=True,
        offset_params=offset_params,
        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop = debug_num_stacks_to_drop
    )

    if not res.is_success:
        raise ApplicationError_NoJobRetrieved(
            domo_instance=auth.domo_instance,
            application_id=application_id,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [10]:
#| eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await get_applications(auth=token_auth)
application = next(
    (app for app in res.response if app["name"] == "Toolkit: DataSet Tag Automation")
)

res = await get_application_jobs(
    application_id=application["applicationId"], auth=token_auth
)
res.response

warning this token has not been validated by who_am_i, run get_auth_token first


[{'jobId': '743c1c6f-80d5-4b47-b02e-0ea28f6a5683',
  'applicationId': 'a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4',
  'customerId': 'mmmm-0012-0200',
  'jobName': 'tag_inactive_owners',
  'jobDescription': 't',
  'userId': 1893952720,
  'executionTimeout': 1440,
  'jobStatus': 'idle',
  'executionPayload': {'emails': ['jae@onyxreporting.com'],
   'resourceTypes': ['datasource'],
   'tagFilter': [],
   'ownerFilter': [],
   'typeFilter': [],
   'domain': None,
   'taggingPolicies': ['ownerInactive'],
   'metricsDatasetId': '29a3417d-1543-46a3-abd1-cbe84f257fb5'},
  'executionResponse': {},
  'shareState': {'sharedEntities': [{'id': '1893952720',
     'type': 'USER',
     'accessLevel': 'OWNER'}]},
  'accounts': [],
  'executionClass': 'com.domo.executor.tags.ResourceTagApplication',
  'created': 1708539691000,
  'updated': 1708980137000,
  'statusChanged': 1711216210000,
  'triggers': [],
  'compressPayload': False}]

In [11]:
# | export
@gd.route_function
async def get_application_job_by_id(
    auth: dmda.DomoFullAuth,
    application_id: str,
    job_id: str,
    parent_class: str = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}/jobs/{job_id}"

    res = await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        session=session,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success:
        raise ApplicationError_NoneRetrieved(
            domo_instance=auth.domo_instance,
            application_id=application_id,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
            job_id=job_id,
        )

    return res

In [12]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

application_id = "a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4"

res = await get_application_jobs(application_id=application_id, auth=token_auth)

job_id = res.response[0]["jobId"]

res = await get_application_job_by_id(
    auth=token_auth, application_id=application_id, job_id=job_id, debug_api=False
)

res.response

warning this token has not been validated by who_am_i, run get_auth_token first


{'jobId': '743c1c6f-80d5-4b47-b02e-0ea28f6a5683',
 'applicationId': 'a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4',
 'customerId': 'mmmm-0012-0200',
 'jobName': 'tag_inactive_owners',
 'jobDescription': 't',
 'userId': 1893952720,
 'executionTimeout': 1440,
 'jobStatus': 'idle',
 'executionPayload': {'emails': ['jae@onyxreporting.com'],
  'resourceTypes': ['datasource'],
  'tagFilter': [],
  'ownerFilter': [],
  'typeFilter': [],
  'domain': None,
  'taggingPolicies': ['ownerInactive'],
  'metricsDatasetId': '29a3417d-1543-46a3-abd1-cbe84f257fb5'},
 'executionResponse': {},
 'executionState': {},
 'shareState': {'sharedEntities': [{'id': '1893952720',
    'type': 'USER',
    'accessLevel': 'OWNER'}]},
 'accounts': [],
 'executionClass': 'com.domo.executor.tags.ResourceTagApplication',
 'created': 1708539691000,
 'updated': 1708980137000,
 'statusChanged': 1711216210000,
 'triggers': [],
 'compressPayload': False}

# Job Configs for Applications


In [13]:
# | export


def generate_remote_domostats(
    target_instance: str,
    report_dict: dict,
    output_dataset_id: str,
    account_id: str,
    schedule_ls: list,
    execution_timeout: int = 1440,
    debug_api: bool = False,
):

    instance_url = f"{target_instance}.domo.com"

    body = {
        "jobName": instance_url,
        "jobDescription": f"Get Remote stat from {instance_url}",
        "executionTimeout": execution_timeout,
        "executionPayload": {
            "remoteInstance": instance_url,
            "policies": report_dict,
            "metricsDatasetId": output_dataset_id,
        },
        "accounts": [account_id],
        "executionClass": "com.domo.executor.subscriberstats.SubscriberStatsExecutor",
        "resources": {"requests": {"memory": "256M"}, "limits": {"memory": "256M"}},
        "triggers": schedule_ls,
    }

    if debug_api:
        pprint(body)

    return body


def generate_body_watchdog_generic(
    job_name: str,
    notify_user_ids_ls: list,
    notify_group_ids_ls: list,
    notify_emails_ls: list,
    log_dataset_id: str,
    schedule_ls: list,
    watchdog_parameter_body: dict,
    execution_timeout=1440,
    debug_api: bool = False,
):

    body = {
        "jobName": job_name,
        "jobDescription": f"Watchdog for {job_name}",
        "executionTimeout": execution_timeout,
        "accounts": [],
        "executionPayload": {
            "notifyUserIds": notify_user_ids_ls or [],
            "notifyGroupIds": notify_group_ids_ls or [],
            "notifyEmailAddresses": notify_emails_ls or [],
            "watcherParameters": watchdog_parameter_body,
            "metricsDatasetId": log_dataset_id,
        },
        "resources": {"requests": {"memory": "256Mi"}, "limits": {"memory": "256Mi"}},
        "triggers": schedule_ls,
    }

    if debug_api:
        pprint(body)

    return body

In [14]:
# | export
class CRUD_ApplicationJob_Error(de.DomoError):
    def __init__(
        self, domo_instance, application_id, message, parent_class, function_name
    ):
        super().__init__(
            self,
            domo_instance=domo_instance,
            entity_id=application_id,
            parent_class=parent_class,
            function_name=function_name,
            message=message,
        )


# create the new RemoteDomostats job


@gd.route_function
async def create_application_job(
    auth: dmda.DomoFullAuth,
    body: dict,
    application_id: str,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    parent_class: str = None,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}/jobs"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        body=body,
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise CRUD_ApplicationJob_Error(
            domo_instance=auth.domo_instance,
            application_id=application_id,
            message=res.response,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res


# update the job
@gd.route_function
async def update_application_job(
    auth: dmda.DomoFullAuth,
    body: dict,
    job_id: str,
    application_id: str,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}/jobs/{job_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise CRUD_ApplicationJob_Error(
            domo_instance=auth.domo_instance,
            application_id=application_id,
            message=res.response,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res


@gd.route_function
async def update_application_job_trigger(
    auth: dmda.DomoFullAuth,
    body: dict,
    job_id: str,
    trigger_id: str,
    application_id: str,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}/jobs/{job_id}/triggers/{trigger_id}"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success:
        raise CRUD_ApplicationJob_Error(
            domo_instance=auth.domo_instance,
            application_id=application_id,
            message=res.response,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [15]:
# | export


@gd.route_function
async def execute_application_job(
    auth: dmda.DomoAuth,
    application_id,
    job_id,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}/jobs/{job_id}/executions"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        body={},
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    return res

In [16]:
#| eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await get_applications(auth=token_auth)
application_id = next(
    (
        app["applicationId"]
        for app in res.response
        if app["name"] == "Toolkit: DataSet Tag Automation"
    )
)

res = await get_application_jobs(application_id=application_id, auth=token_auth)
job_id = next((job["jobId"] for job in res.response))

job_id

await execute_application_job(
    auth=token_auth, application_id=application_id, job_id=job_id
)

warning this token has not been validated by who_am_i, run get_auth_token first


ResponseGetData(status=200, response={'requestedBy': 1893952720, 'queuedJobs': [{'jobId': '743c1c6f-80d5-4b47-b02e-0ea28f6a5683', 'applicationId': 'a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4', 'executionId': '7b8ba4d6-f803-4d98-94d5-570be0c68c68', 'customerId': 'mmmm-0012-0200', 'jobName': 'tag_inactive_owners', 'jobDescription': 't', 'userId': 1893952720, 'executionTimeout': 1440, 'jobStatus': 'idle', 'executionPayload': {'emails': ['jae@onyxreporting.com'], 'resourceTypes': ['datasource'], 'tagFilter': [], 'ownerFilter': [], 'typeFilter': [], 'domain': None, 'taggingPolicies': ['ownerInactive'], 'metricsDatasetId': '29a3417d-1543-46a3-abd1-cbe84f257fb5'}, 'executionResponse': {}, 'accounts': [], 'executionClass': 'com.domo.executor.tags.ResourceTagApplication', 'created': 1708539691000, 'updated': 1708980137000, 'statusChanged': 1711216210000, 'triggers': [], 'compressPayload': False}]}, is_success=True, parent_class=None)

In [17]:
# | hide
import nbdev

nbdev.nbdev_export("./application.ipynb")
!nbqa black application.ipynb

All done! ✨ 🍰 ✨
1 file left unchanged.
